In [127]:
import dask.dataframe as dd
from dask.distributed import Client
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

In [10]:
LobNamevar=input("Enter the LOBNAME")
speriod=int(input("Enter the simulation period: "))
samples=int(input("Enter the number of samples: "))

In [163]:


# Define the folder containing the Parquet files
folder_path = r'D:\RISHIN\13_ILC_TASK1\input\PARQUET_FILES'

# List all Parquet files in the folder
parquet_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Initialize an empty DataFrame to store the results
final_grouped_df = pd.DataFrame()

# Process each Parquet file individually
for file in parquet_files:
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file)
    
    # Convert the PyArrow Table to a Pandas DataFrame
    df = table.to_pandas()
    
    # Filter the DataFrame where LobName equals LobNamevar
    filtered_df = df[df['LobName'] == LobNamevar]
    
    # Perform the aggregation: sum the Loss column grouped by EventId and PeriodId
    grouped_df = filtered_df.groupby(['EventId', 'PeriodId'], as_index=False).agg({'Loss': 'sum'})
    
    # Rename the aggregated column to Sum_Loss
    grouped_df.rename(columns={'Loss': 'Sum_Loss'}, inplace=True)
    
    # Append the grouped DataFrame to the final_grouped_df
    final_grouped_df = pd.concat([final_grouped_df, grouped_df], ignore_index=True)

# Perform final grouping and sorting
final_grouped_df = final_grouped_df.groupby(['EventId', 'PeriodId'], as_index=False).agg({'Sum_Loss': 'sum'})
final_grouped_df = final_grouped_df.sort_values(by=[ 'Sum_Loss'], ascending=[ False])

# The DataFrame is now ready for the next instructions
dataframe_1 = final_grouped_df

In [165]:
# Initialize dataframe_2 by selecting PeriodId and max(Sum_Loss) grouped by PeriodId
dataframe_2 = dataframe_1.groupby('PeriodId', as_index=False).agg({'Sum_Loss': 'max'})

# Rename the aggregated column to Max_Loss
dataframe_2.rename(columns={'Sum_Loss': 'Max_Loss'}, inplace=True)

# Sort dataframe_2 by Max_Loss in descending order
dataframe_2 = dataframe_2.sort_values(by='Max_Loss', ascending=False)



In [166]:
# Initialize dataframe_2 by selecting PeriodId and Sum(Sum_Loss) grouped by PeriodId
dataframe_3 = dataframe_1.groupby('PeriodId', as_index=False).agg({'Sum_Loss': 'sum'})

# Rename the aggregated column to Sum_Loss
dataframe_3.rename(columns={'Sum_Loss': 'S_Sum_Loss'}, inplace=True)

# Sort dataframe_3 by S_sum_Loss in descending order
dataframe_3 = dataframe_3.sort_values(by='S_Sum_Loss', ascending=False)



In [178]:
#dataframe_2['Max_Loss'] = dataframe_2['Max_Loss'].round(5)

dataframe_2['rate'] = (1 / (speriod * samples))

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_2['cumrate'] = dataframe_2['rate'].cumsum()

# Calculate the RPs column and round to 6 decimal places
dataframe_2['RPs'] = (1 / dataframe_2['cumrate'])

# Calculate the TCE_OEP_1 column and round to 6 decimal places
dataframe_2['TCE_OEP_1'] = ((dataframe_2['Max_Loss'] - dataframe_2['Max_Loss'].shift(-1)) * 
                          (dataframe_2['cumrate'] + dataframe_2['cumrate'].shift(-1)) * 0.5)

# Calculate the TCE_OEP_2 column and round to 6 decimal places
dataframe_2['TCE_OEP_2'] = (dataframe_2['TCE_OEP_1'].shift().cumsum() * dataframe_2['RPs']).round(6)


# Calculate the TCE_OEP_Final column and round to 6 decimal places
dataframe_2['TCE_OEP_Final'] = (dataframe_2['TCE_OEP_2'] + dataframe_2['Max_Loss'])

In [179]:
#dataframe_3['S_Sum_Loss'] = dataframe_3['S_Sum_Loss'].round(5)

# Calculate the rate column and round to 6 decimal places
dataframe_3['rate'] = (1 / (speriod * samples))

# Calculate the cumulative rate column and round to 6 decimal places
dataframe_3['cumrate'] = dataframe_3['rate'].cumsum()

# Calculate the RPs column and round to 6 decimal places
dataframe_3['RPs'] = (1 / dataframe_3['cumrate'])
# Calculate the TCE_AEP_1 column and round to 6 decimal places
dataframe_3['TCE_AEP_1'] = ((dataframe_3['S_Sum_Loss'] - dataframe_3['S_Sum_Loss'].shift(-1)) * 
                          (dataframe_3['cumrate'] + dataframe_3['cumrate'].shift(-1)) * 0.5)

# Calculate the cumulative sum up to the previous row and multiply by the current row's RPs, then round to 6 decimal places
dataframe_3['TCE_AEP_2'] = (dataframe_3['TCE_AEP_1'].shift().cumsum() * dataframe_3['RPs']).round(6)

# Calculate the TCE_AEP_Final column and round to 6 decimal places
dataframe_3['TCE_AEP_Final'] = (dataframe_3['TCE_AEP_2'] + dataframe_3['S_Sum_Loss'])

In [180]:
dataframe_2.head(20)

,PeriodId,Max_Loss,rate,cumrate,RPs,TCE_OEP_1,TCE_OEP_2,TCE_OEP_Final
147788,149148,1654085153.7900753021,0.0000040000,0.0000040000,250000.0000000000,869.9915659151,NaN,NaN
197324,199148,1509086559.4708862305,0.0000040000,0.0000080000,125000.0000000000,1496.2968361422,108748945.7393919975,1617835505.2102782726
205656,207557,1359456875.8566699028,0.0000040000,0.0000120000,83333.3333333333,559.1564880431,197190700.1714409888,1556647576.0281109810
4264,4312,1319517126.7107307911,0.0000040000,0.0000160000,62500.0000000000,1194.4848863135,182840305.6312780082,1502357432.3420088291
152903,154312,1253156855.2488694191,0.0000040000,0.0000200000,50000.0000000000,1155.4753544021,205996488.8206979930,1459153344.0695674419
103347,104312,1200635248.2305915356,0.0000040000,0.0000240000,41666.6666666667,423.5720373477,219808547.1173360050,1420443795.3479275703
191303,193067,1184344016.0249090195,0.0000040000,0.0000280000,35714.2857142857,974.1932803360,203534898.8629930019,1387878914.8879020214
48693,49148,1151870906.6803750992,0.0000040000,0.0000320000,31250.0000000000,552.3813139309,208536576.5156199932,1360407483.1959950924
98237,99148,1135624397.4471125603,0.0000040000,0.0000360000,27777.7777777778,1196.0639068495,200709771.1786319911,1336334168.6257445812
236882,239080,1104149031.4773890972,0.0000040000,0.0000400000,25000.0000000000,173.0047846019,210540391.7320060134,1314689423.2093951702


In [181]:
dataframe_3[dataframe_3['PeriodId'] == 76654]

,PeriodId,S_Sum_Loss,rate,cumrate,RPs,TCE_AEP_1,TCE_AEP_2,TCE_AEP_Final
75941,76654,286667534.2583946586,0.0000040000,0.0025200000,396.8253968254,158.0808693056,149690203.8880729973,436357738.1464676857


In [172]:
pd.options.display.float_format = '{:.10f}'.format
# Revert the display format to the default settings
#pd.reset_option('display.float_format')

In [182]:
dataframe_2[dataframe_2['PeriodId'] ==99346]

,PeriodId,Max_Loss,rate,cumrate,RPs,TCE_OEP_1,TCE_OEP_2,TCE_OEP_Final
98434,99346,377800122.4189433455,0.0000040000,0.0010000000,1000.0000000000,453.1942736338,185139509.6673130095,562939632.0862563848


In [138]:
dataframe_2.to_csv(r'D:\RISHIN\13_ILC_TASK1\output\TCE_OEP.csv', index=False)

In [ ]:
# similary i want an another script..so in first case is 1) select PeriodId,EventId,sum(Loss)Sum_Loss where LobName == LobNamevar grouped by PeriodId,EventId ordered by sum_loss DESC
# 2) select PeriodId,max(Loss)Max_Loss grouped by PeriodId  ordered by Max_Loss DESC from the result of first case 

# I will give the third case later